# Morningstar Pro - Entraînement avancé sur Colab

## Système complet de trading algorithmique avec données sociales

Ce notebook permet :
- De télécharger les données de marché (OHLCV) depuis un exchange crypto
- D'ajouter des indicateurs techniques avancés
- D'intégrer des données sociales (GitHub et Reddit)
- D'entraîner un modèle de deep learning pour le trading

## 1. Installation des dépendances

In [ ]:
# Installation des dépendances système et Python
# Forcer la réinstallation de TF et NumPy (version 1.23.5) pour compatibilité
!pip install --force-reinstall -q tensorflow==2.12.0 numpy==1.23.5
!pip install -q pandas==1.5.3 ccxt==4.1.91 ta pyarrow scikit-learn asyncpraw tweepy aiohttp PyGithub praw nest_asyncio

# --- IMPORTANT : Redémarrez l'environnement d'exécution après cette cellule --- #
# (Menu Exécution > Redémarrer l'environnement d'exécution)
import os
os.kill(os.getpid(), 9)

## 2. Configuration

In [ ]:
# Clonage du dépôt et ajout du chemin Morningstar
!git clone https://github.com/Cabrel10/eva001.git
import sys
sys.path.insert(0, '/content/eva001')

In [ ]:
# Sélection interactive des paires et des dates
import datetime
default_pairs = 'BTC/USDT,ETH/USDT,BNB/USDT,SOL/USDT'
pairs = input(f"Entrez les paires séparées par une virgule (exemple: {default_pairs}): ") or default_pairs
pairs = [p.strip() for p in pairs.split(',')]
start_date = input("Date de début (YYYY-MM-DD, défaut 2023-01-01): ") or '2023-01-01'
end_date = input("Date de fin (YYYY-MM-DD, défaut aujourd'hui): ") or str(datetime.date.today())

In [ ]:
# Configuration des APIs sociales
from github import Github
import praw

# Config GitHub (remplacer par ton token)
github_token = input("Entrez votre token GitHub (ou laissez vide pour désactiver): ") or None
gh = Github(github_token) if github_token else None

# Config Reddit (remplacer par tes credentials, optionnel)
reddit_client_id = input("Reddit client_id (optionnel): ") or None
reddit_client_secret = input("Reddit client_secret (optionnel): ") or None
reddit = None
if reddit_client_id and reddit_client_secret:
    try:
        reddit = praw.Reddit(
            client_id=reddit_client_id,
            client_secret=reddit_client_secret,
            user_agent="Morningstar Data Collector",
            read_only=True # Mode lecture seule
        )
        print("Reddit API configurée.")
    except Exception as e:
        print(f"Erreur configuration Reddit: {e}. Les données Reddit ne seront pas collectées.")
else:
    print("Credentials Reddit non fournis. Les données Reddit ne seront pas collectées.")

## 3. Pipeline de données

In [ ]:
# Fonctions pour données sociales
def get_github_stats(repo_name):
    if not gh:
        return None, None, None, None, None
    try:
        repo = gh.get_repo(repo_name)
        return (
            repo.get_commits().totalCount,
            repo.stargazers_count,
            repo.forks_count,
            repo.get_issues(state='open').totalCount,
            repo.get_issues(state='closed').totalCount
        )
    except Exception as e:
        print(f"Erreur GitHub pour {repo_name}: {e}")
        return None, None, None, None, None

def get_reddit_sentiment(subreddit, pair):
    if not reddit or not reddit.read_only:
        print("Reddit non configuré ou non en lecture seule.")
        return None
    try:
        submissions = reddit.subreddit(subreddit).search(f"{pair} flair:Discussion", limit=10)
        scores = [s.score for s in submissions]
        if not scores:
             return None # Pas de posts trouvés
        positive_ratio = sum(1 for s in scores if s > 0) / len(scores)
        return positive_ratio
    except Exception as e:
        print(f"Erreur Reddit pour {subreddit} / {pair}: {e}")
        return None

In [ ]:
# Téléchargement des données de marché
from Morningstar.utils.data_manager import ExchangeDataManager
import pandas as pd
import asyncio
import nest_asyncio

nest_asyncio.apply()

async def fetch_data(pairs, timeframe='1h', start_date=None, end_date=None):
    exchange = ExchangeDataManager(exchange_name="kucoin")
    await exchange.load_markets_async()
    all_data = []
    tasks = []
    for pair in pairs:
        print(f"Préparation du téléchargement {pair}...")
        tasks.append(exchange.load_data(pair, timeframe, start_date, end_date))
    
    results = await asyncio.gather(*tasks, return_exceptions=True)
    await exchange.close()

    for i, result in enumerate(results):
        pair = pairs[i]
        if isinstance(result, pd.DataFrame) and not result.empty:
            print(f"Données {pair} téléchargées.")
            result['pair'] = pair
            all_data.append(result)
        else:
            print(f"Échec ou données vides pour {pair}: {result}")
            
    if all_data:
        return pd.concat(all_data)
    else:
        raise ValueError("Aucune donnée téléchargée pour aucune paire.")

raw_data = asyncio.get_event_loop().run_until_complete(fetch_data(pairs, '1h', start_date, end_date))

In [ ]:
# Prétraitement et sauvegarde
from Morningstar.utils.custom_indicators import add_technical_indicators

def prepare_dataset(df):
    # Réinitialiser l'index pour éviter les doublons
    df = df.reset_index(drop=True)
    
    # Ajouter les indicateurs techniques
    df = add_technical_indicators(df)
    
    # Récupérer les données sociales pour chaque paire
    for pair in df['pair'].unique():
        # Mappage des paires aux repos GitHub et subreddits
        repo_map = {
            'BTC/USDT': 'bitcoin/bitcoin',
            'ETH/USDT': 'ethereum/go-ethereum',
            'BNB/USDT': 'binance-chain/docs',
            'SOL/USDT': 'solana-labs/solana'
        }
        
        subreddit_map = {
            'BTC/USDT': 'Bitcoin',
            'ETH/USDT': 'ethereum',
            'BNB/USDT': 'binance',
            'SOL/USDT': 'solana'
        }
        
        mask = df['pair'] == pair
        
        if pair in repo_map:
            print(f"Récupération GitHub pour {pair}...")
            commits, stars, forks, issues_opened, issues_closed = get_github_stats(repo_map[pair])
            df.loc[mask, 'commits'] = commits
            df.loc[mask, 'stars'] = stars
            df.loc[mask, 'forks'] = forks
            df.loc[mask, 'issues_opened'] = issues_opened
            df.loc[mask, 'issues_closed'] = issues_closed
            
        if pair in subreddit_map:
            print(f"Récupération Reddit pour {pair}...")
            sentiment = get_reddit_sentiment(subreddit_map[pair], pair.split('/')[0])
            df.loc[mask, 'reddit_sentiment'] = sentiment
    
    # Colonnes finales
    columns = [
        'open', 'high', 'low', 'close', 'volume', 'rsi', 'macd', 'macd_signal', 'macd_hist',
        'bb_upper', 'bb_middle', 'bb_lower', 'volume_ma', 'volume_anomaly', 'pair',
        'commits', 'stars', 'forks', 'issues_opened', 'issues_closed', 'reddit_sentiment', 'datetime'
    ]
    
    # S'assurer que toutes les colonnes existent et remplir les NaN
    for col in columns:
        if col not in df.columns:
            df[col] = None
    df = df[columns] # Réorganiser
    df = df.fillna(method='ffill').fillna(0) # Remplir les NaN restants
            
    return df

data = prepare_dataset(raw_data.copy()) # Utiliser une copie pour éviter les modifs inplace
data.to_parquet('full_dataset.parquet')
print(f"Dataset final: {data.shape}")
print(data.head())

## 4. Entraînement du modèle

In [ ]:
# Entraînement du modèle Morningstar
import tensorflow as tf
from Morningstar.workflows.training_workflow import TrainingWorkflow
class ColabConfig:
    def __init__(self):
        self.time_window = 50
        # S'assurer que 'datetime' n'est pas dans les features si c'est l'index
        self.features = [col for col in data.columns if col != 'datetime'] 
        self.epochs = 200
        self.batch_size = 1024
        self.dataset_path = 'full_dataset.parquet'
colab_config = ColabConfig()
workflow = TrainingWorkflow(colab_config)

# Préparer le dataset TensorFlow
# Assurer que 'data' est le DataFrame final préparé
tf_dataset = workflow._prepare_dataset(data) 

dataset_size = tf.data.experimental.cardinality(tf_dataset).numpy()
if dataset_size <= 1:
    raise ValueError("Le dataset est trop petit ou vide pour l'entraînement.")
    
val_size = int(dataset_size * 0.2)
if val_size == 0 and dataset_size > 1:
    val_size = 1 # Assurer au moins 1 élément pour la validation si possible

train_dataset = tf_dataset.skip(val_size)
val_dataset = tf_dataset.take(val_size)

# Vérifier que les datasets ne sont pas vides
if tf.data.experimental.cardinality(train_dataset).numpy() == 0:
    raise ValueError("Le dataset d'entraînement est vide.")
if tf.data.experimental.cardinality(val_dataset).numpy() == 0:
    print("Attention: Le dataset de validation est vide. L'entraînement se fera sans validation.")
    val_dataset = None # Entraîner sans validation si vide

with tf.distribute.MirroredStrategy().scope():
    # Définir l'input shape basé sur les features réelles
    input_shape = (colab_config.time_window, len(colab_config.features))
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Conv1D(128, 5, activation='swish')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LSTM(256, return_sequences=True)(x)
    x = tf.keras.layers.LSTM(128)(x)
    x = tf.keras.layers.Dense(64, activation='swish')(x)
    outputs = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='huber',
        metrics=['mae']
    )
model.summary()

callbacks = [
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss' if val_dataset else 'loss'),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5, monitor='val_loss' if val_dataset else 'loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss' if val_dataset else 'loss', restore_best_weights=True) # Ajout EarlyStopping
]

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=colab_config.epochs,
    batch_size=colab_config.batch_size, # Batch size est géré par .batch() sur le dataset tf
    callbacks=callbacks
)

## 5. Sauvegarde finale

In [ ]:
# Sauvegarde finale et export sur Google Drive
model.save('morningstar_pro.h5')
from google.colab import drive
drive.mount('/content/drive')
!cp morningstar_pro.h5 '/content/drive/MyDrive/Colab Data/'